In [123]:
import numpy as np 
import pandas as pd
from statsmodels import api as sm
from scipy import stats

# data

In [124]:
# data = pd.read_pickle("./data/final_data_panel.pkl").rename(columns={"Feps":"disp"})
# data = pd.read_pickle("./data/final_data_panel2.pkl").drop(columns=["ref_price"]).rename(columns={"Feps":"disp"})
# data = pd.read_pickle("./data/final_data_panel_version3.pkl").drop(columns=["ref_price", "price"])
data = pd.read_pickle("./final_data_panel_proxies_window30month.pkl")
# data.head()
data.columns

other_factor_cols = ['cgo', 'LOGBM',
       'LOGME', 'mom_minus1_0', 'mom_minus12_minus1', 'mom_minus36_minus12',
       'mom_minus11_minus2', "turnover"]
proxy_cols = ['beta_MKT', 'ivol', 'retvol', 'age', 'disp', "CFVOL"]
# data = data.drop(columns="turnover")
data.head()# data.columns

,date,asset,beta_MKT,ivol,retvol,age,disp,CFVOL,LOGBM,LOGME,mom_minus1_0,mom_minus12_minus1,mom_minus36_minus12,mom_minus11_minus2,cgo,turnover,future_return,score
0,2014-01,000006,0.714892,-0.385213,-0.253707,1.623687,NaN,-0.195242,0.852785,-0.290561,-1.010538,-0.434441,1.405009,0.228706,-0.437893,-0.381935,-0.016018,-1.735858
1,2014-01,000021,0.902618,-0.210387,0.403084,1.371718,NaN,0.330226,0.302500,0.840230,2.201666,0.015993,-1.109247,0.205026,0.858371,-0.227697,-0.077961,-0.956005
2,2014-01,000028,-0.789488,0.177394,0.003037,1.455708,NaN,-0.804418,-1.637611,1.817995,1.169076,0.485133,1.219088,0.203879,2.514612,-0.564752,-0.078858,1.029076
3,2014-01,000030,-0.926812,3.139430,3.924975,1.441709,NaN,0.162950,0.281369,0.279086,-0.336719,-1.182369,0.487879,-1.298161,-0.311534,-0.033620,0.255537,1.454450
4,2014-01,000031,0.503439,-0.779173,-0.700685,1.427711,NaN,-0.857062,0.960635,-0.096190,-0.409915,-0.677725,-0.352625,-0.694764,-0.952216,-0.911117,-0.090141,-0.388839


In [125]:
rp = pd.read_parquet("./rp.parquet")
# rp.head()
# rp.describe()
rp["cgo"] = (rp["price"] - rp["rp"]) / rp["price"]
rp["time"] = pd.to_datetime(rp["date"]).dt.to_period("M")
cgo = rp.groupby(["time", "id"], group_keys=False)["cgo"].mean().reset_index()
cgo.tail()
cgo.describe()
cgo["cgo"].max()

# 用不了，数据偏差巨大

np.float64(1.5869471170143748e+93)

In [126]:
# tvr = pd.read_pickle("./data/daily_turnover.pkl").reset_index()
# tvr = tvr[pd.to_datetime(tvr["tradedate"]) > "2013-10-01"]
# tvr["asset"] = tvr["order_book_id"].str[:6]
# tvr["date"] = pd.to_datetime(tvr["tradedate"]).dt.to_period("M")
# tvr = tvr.drop(columns=["order_book_id", "tradedate"]).rename(columns={"today":"turnover"})

# # tvr = tvr.groupby(["date", "asset"]).sum().reset_index()
# tvr = tvr.groupby(["date", "asset"]).mean().reset_index()

# tvr.head()

In [127]:
# data = pd.merge(data, tvr, on=["date", "asset"], how="left")
# data.head(3)

In [128]:
# data.to_pickle("./data/final_data_panel_version6.pkl")

In [129]:
# 算score代码，dbs算，弃用

# def grouped_qcut(series, q=101):
#     out = pd.Series(np.nan, index=series.index)
#     valid = series.dropna()
#     out.loc[valid.index] = pd.qcut(valid, q, labels=False, duplicates='drop')
#     return out

# data['rank_LOGBM'] = data.groupby('date')['LOGBM'].transform(lambda s: grouped_qcut(s, 101))
# data['rank_Mom_minus11_minus_2'] = data.groupby('date')['mom_minus11_minus2'].transform(lambda s: grouped_qcut(s, 101))
# # data['rank_LOGBM'] = data.groupby('date')['LOGBM'].transform(lambda s: s.rank())
# # data['rank_Mom_minus11_minus_2'] = data.groupby('date')['mom_minus11_minus2'].transform(lambda s: s.rank())
# data['score'] = (data['rank_LOGBM'] + data['rank_Mom_minus11_minus_2'])/2
# data.drop(columns=["rank_LOGBM", "rank_Mom_minus11_minus_2"], inplace=True)
# data.head()


##### 第二版
# def grouped_qcut(series, q=101):
#     out = pd.Series(np.nan, index=series.index)
#     valid = series.dropna()
#     out.loc[valid.index] = pd.qcut(valid, q, labels=False, duplicates='raise')
#     return out
# # 假设高 LOGBM 是好股票（被低估），那么它应该有低的 Mispricing Score
# data['rank_LOGBM'] = data.groupby('date')['LOGBM'].transform(
#     lambda s: 100 - grouped_qcut(s, 101) # 反向
# )
# # 动量同理，如果是 Winners (高动量) 预期收益高，则应对应低 Mispricing
# data['rank_Mom_minus11_minus_2'] = data.groupby('date')['mom_minus11_minus2'].transform(
#     lambda s: 100 - grouped_qcut(s, 101)
# )
# data['score'] = (data['rank_LOGBM'] + data['rank_Mom_minus11_minus_2'])/2

In [130]:
# 全换成rank
data2 = data.copy()#[["date", "asset"]+proxy_cols]

# for p in proxy_cols:
#     data2[p] = data2.groupby("date", group_keys=False)[p].transform(lambda x: x.rank())

# 对非proxy的列standardize
# for q in 

data2.head()

,date,asset,beta_MKT,ivol,retvol,age,disp,CFVOL,LOGBM,LOGME,mom_minus1_0,mom_minus12_minus1,mom_minus36_minus12,mom_minus11_minus2,cgo,turnover,future_return,score
0,2014-01,000006,0.714892,-0.385213,-0.253707,1.623687,NaN,-0.195242,0.852785,-0.290561,-1.010538,-0.434441,1.405009,0.228706,-0.437893,-0.381935,-0.016018,-1.735858
1,2014-01,000021,0.902618,-0.210387,0.403084,1.371718,NaN,0.330226,0.302500,0.840230,2.201666,0.015993,-1.109247,0.205026,0.858371,-0.227697,-0.077961,-0.956005
2,2014-01,000028,-0.789488,0.177394,0.003037,1.455708,NaN,-0.804418,-1.637611,1.817995,1.169076,0.485133,1.219088,0.203879,2.514612,-0.564752,-0.078858,1.029076
3,2014-01,000030,-0.926812,3.139430,3.924975,1.441709,NaN,0.162950,0.281369,0.279086,-0.336719,-1.182369,0.487879,-1.298161,-0.311534,-0.033620,0.255537,1.454450
4,2014-01,000031,0.503439,-0.779173,-0.700685,1.427711,NaN,-0.857062,0.960635,-0.096190,-0.409915,-0.677725,-0.352625,-0.694764,-0.952216,-0.911117,-0.090141,-0.388839


In [131]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57403 entries, 0 to 57402
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype    
---  ------               --------------  -----    
 0   date                 57403 non-null  period[M]
 1   asset                57403 non-null  object   
 2   beta_MKT             55084 non-null  float64  
 3   ivol                 49862 non-null  float64  
 4   retvol               57403 non-null  float64  
 5   age                  57403 non-null  float64  
 6   disp                 54901 non-null  float64  
 7   CFVOL                54697 non-null  float64  
 8   LOGBM                57396 non-null  float64  
 9   LOGME                57402 non-null  float64  
 10  mom_minus1_0         57403 non-null  float64  
 11  mom_minus12_minus1   57403 non-null  float64  
 12  mom_minus36_minus12  54131 non-null  float64  
 13  mom_minus11_minus2   57403 non-null  float64  
 14  cgo                  57130 non-null  float64  
 15  tu

# fama-macbeth

In [132]:
def calculate_newey_west_t_stat(data, lag_type='auto', kernel='bartlett'):
    """
    计算Newey-West调整的t统计量（完整版）
    
    Parameters:
    -----------
    data : array-like
        时间序列数据
    lag_type : str or int
        'auto': 自动选择滞后阶数
        int: 指定滞后阶数
    kernel : str
        核函数类型：'bartlett', 'uniform', 'parzen'等
        
    Returns:
    --------
    dict: 包含t统计量、标准误、均值等信息的字典
    """
    data = np.asarray(data).flatten()
    n = len(data)
    
    if n <= 1:
        return {
            't_stat': np.nan,
            'se': np.nan,
            'mean': np.nan if n == 0 else data[0],
            'nobs': n
        }
    
    mean_val = np.mean(data)
    residuals = data - mean_val
    
    # 确定滞后阶数
    if lag_type == 'auto':
        # Andrews (1991) 或 Newey-West (1994) 建议的滞后选择
        max_lags = int(np.floor(4 * (n / 100) ** (2 / 9)))
    else:
        max_lags = int(lag_type)
    
    max_lags = min(max_lags, n - 1)
    
    # 计算自协方差
    autocov = [np.mean(residuals ** 2)]  # γ(0)
    for lag in range(1, max_lags + 1):
        autocov.append(np.mean(residuals[lag:] * residuals[:-lag]))
    
    # 计算核权重
    weights = []
    for lag in range(max_lags + 1):
        if kernel == 'bartlett':
            weight = 1 - lag / (max_lags + 1) if lag <= max_lags else 0
        elif kernel == 'uniform':
            weight = 1.0
        elif kernel == 'parzen':
            z = lag / (max_lags + 1)
            if z <= 0.5:
                weight = 1 - 6 * z**2 + 6 * z**3
            else:
                weight = 2 * (1 - z)**3
        else:
            weight = 1 - lag / (max_lags + 1)  # 默认Bartlett核
        weights.append(weight)
    
    # 计算Newey-West方差估计
    var_nw = autocov[0] * weights[0]
    for lag in range(1, max_lags + 1):
        var_nw += 2 * weights[lag] * autocov[lag]
    
    # 调整样本量
    var_nw = var_nw / n
    
    # 计算标准误和t统计量
    if var_nw > 0:
        se_nw = np.sqrt(var_nw)
        t_stat_nw = mean_val / se_nw
    else:
        se_nw = np.nan
        t_stat_nw = np.nan
    
    return {
        't_stat': t_stat_nw,
        'se': se_nw,
        'mean': mean_val,
        'nobs': n,
        'lags_used': max_lags,
        'variance_nw': var_nw
    }

In [133]:
def famaMacBeth(df: pd.DataFrame, var_cols: list, time_col="date", asset_col="asset", y_col: str = "future_return"):
    """
    执行Fama-MacBeth回归
    
    参数:
    ----------
    df : pd.DataFrame
        面板数据，必须包含以下列：
        - time: 时间标识（如月份）
        - id: 证券标识符
        - y_col: 因变量（默认为"ret"）
        - var_cols列表中的所有自变量
    var_cols : list
        自变量（因子/特征）列名的列表
    y_col : str
        因变量的列名，默认为"ret"
        
    返回:
    ----------
    dict: 包含以下键的字典:
        - 'risk_premia': 因子溢价结果DataFrame
        - 'lambda_ts': 各期lambda系数的时间序列DataFrame
        - 'rsquared': 各期回归的R²时间序列
        - 'summary': 文本格式的汇总统计
    """
    
    # 复制数据以避免修改原始数据
    data = df.copy()
    
    # 确保数据按时间排序
    data = data.sort_values([time_col, asset_col])
    
    # 获取所有时间点
    time_periods = sorted(data[time_col].unique())
    
    # 存储结果的容器
    lambda_results = []  # 各期的lambda系数
    rsquared_results = []  # 各期的R²
    tstats_results = []  # 各期的t统计量
    
    # 第一步：对每个时间点进行横截面回归
    # print("开始第一步：月度横截面回归...")
    for t in time_periods:
        # 获取当前时间点的横截面数据
        cross_section = data[data[time_col] == t].copy()
        
        # 删除当前回归中用到的变量中有缺失值的行
        regression_vars = [y_col] + var_cols
        cross_section = cross_section.dropna(subset=regression_vars)
        
        # 如果数据不足，跳过该期
        if len(cross_section) < len(var_cols) + 1:
            print(f"时间点 {t}: 样本量不足 ({len(cross_section)})，跳过该期")
            continue
            
        # 准备自变量和因变量
        X = cross_section[var_cols].values
        y = cross_section[y_col].values
        
        # 添加常数项
        X = sm.add_constant(X)
        
        # 执行OLS回归
        try:
            model = sm.OLS(y, X, missing='drop')
            results = model.fit()
            
            # 保存结果
            lambda_t = results.params  # 系数
            tstats_t = results.tvalues  # t统计量
            rsquared_t = results.rsquared  # R²
            
            # 将结果添加到列表
            lambda_results.append(lambda_t)
            tstats_results.append(tstats_t)
            rsquared_results.append(rsquared_t)
            
            # 打印进度（每12期打印一次）
            # if len(lambda_results) % 12 == 0:
                # print(f"已完成 {len(lambda_results)} 期回归")
                
        except Exception as e:
            print(f"时间点 {t}: 回归失败 - {str(e)}")
            continue
    
    # 如果没有任何成功的回归，返回空结果
    if len(lambda_results) == 0:
        print("错误: 没有成功完成任何一期的回归")
        return None
    
    # 将结果转换为DataFrame
    lambda_df = pd.DataFrame(lambda_results)
    lambda_df.columns = ['intercept'] + var_cols
    
    # 将时间点添加到lambda_df
    successful_periods = [time_periods[i] for i in range(len(time_periods)) 
                         if i < len(lambda_results)]  # 只包含成功的期数
    lambda_df[time_col] = successful_periods
    lambda_df = lambda_df.set_index(time_col)
    
    # 第二步：计算因子溢价（时间序列平均）
    # print("\n开始第二步：计算因子溢价（时间序列平均）...")
    
    # 计算每列（每个因子）的时间序列均值、标准差、t统计量和p值
    risk_premia = pd.DataFrame(index=['intercept'] + var_cols)
    
    for col in ['intercept'] + var_cols:
        # 获取该因子的时间序列系数
        lambda_ts = lambda_df[col].values
        
        # 计算统计量
        mean = np.mean(lambda_ts)  # 因子溢价
        std = np.std(lambda_ts, ddof=1)  # 标准差
        n = len(lambda_ts)  # 期数
        
        # 计算t统计量和p值
        # t_stat = mean / (std / np.sqrt(n)) if std > 0 else np.nan
        t_stat = calculate_newey_west_t_stat(lambda_ts)["t_stat"]
        p_value = 2 * (1 - stats.t.cdf(abs(t_stat), n-1)) if not np.isnan(t_stat) else np.nan
        
        # 存储结果
        risk_premia.loc[col, 'mean'] = mean
        risk_premia.loc[col, 'std'] = std
        risk_premia.loc[col, 't_stat'] = t_stat
        risk_premia.loc[col, 'p_value'] = p_value
        risk_premia.loc[col, 'n_periods'] = n
        
        # 计算95%置信区间
        ci_lower = mean - 1.96 * (std / np.sqrt(n))
        ci_upper = mean + 1.96 * (std / np.sqrt(n))
        risk_premia.loc[col, 'ci_95_lower'] = ci_lower
        risk_premia.loc[col, 'ci_95_upper'] = ci_upper
    
    # 创建汇总文本
    summary = "Fama-MacBet reg result\n"
    summary += "=" * 60 + "\n"
    summary += f"# sample period: {len(lambda_results)}\n"
    summary += f"dep var: {y_col}\n"
    summary += f"ind var: {', '.join(var_cols)}\n"
    summary += "=" * 60 + "\n\n"
    summary += "factor premium est.:\n"
    summary += "-" * 80 + "\n"
    
    # 格式化输出
    for idx, row in risk_premia.iterrows():
        significance = "***" if row['p_value'] < 0.01 else "**" if row['p_value'] < 0.05 else "*" if row['p_value'] < 0.1 else ""
        summary += f"{idx:15s}: {row['mean']:.6f} (t={row['t_stat']:.3f}){significance}\n"
    
    summary += "\n" + "-" * 80 + "\n"
    summary += "Significance: * p<0.1, ** p<0.05, *** p<0.01\n"
    
    # 打印摘要
    # print(summary)
    
    # 准备返回结果
    # results = {
    #     'risk_premia': risk_premia,  # 因子溢价结果
    #     'lambda_ts': lambda_df,  # 各期lambda系数的时间序列
    #     'rsquared': pd.Series(rsquared_results, index=successful_periods),  # 各期R²
    #     'summary': summary  # 文本摘要
    # }
    results = summary
    return results

In [134]:
# famaMacBeth(data, other_factor_cols, y_col="future_return")

# 3.1 regression

In [135]:
result_list_3_1 = []

df = data2.copy()

# calc results
for proxy in proxy_cols:
    data5 = df[["date", "asset", "future_return"] + other_factor_cols].copy()
    data5["proxy"] = df[proxy]
    data5["proxy*cgo"] = data5["proxy"] * data5["cgo"]
    # data5["mom_12_1*proxy"] = data5["mom_minus12_minus1"] * data5["proxy"]
    # data3[""]
    var_cols = data5.drop(columns=["date", "asset", "future_return"]).columns.to_list()
    # print(var_cols)
    
    result_list_3_1.append(famaMacBeth(data5, var_cols))
    # break

# then print results

In [136]:
i = 0
print(proxy_cols[i])
print(result_list_3_1[i])

beta_MKT
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.006889 (t=1.060)
cgo            : 0.000549 (t=0.547)
LOGBM          : 0.000629 (t=0.411)
LOGME          : -0.002756 (t=-2.645)***
mom_minus1_0   : 0.000154 (t=0.117)
mom_minus12_minus1: 0.002142 (t=0.965)
mom_minus36_minus12: -0.001263 (t=-1.451)
mom_minus11_minus2: -0.000028 (t=-0.015)
turnover       : -0.004790 (t=-4.163)***
proxy          : -0.000039 (t=-0.045)
proxy*cgo      : -0.000213 (t=-0.443)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [137]:
i = 1
print(proxy_cols[i])
print(result_list_3_1[i])

ivol
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.007094 (t=1.085)
cgo            : 0.000033 (t=0.029)
LOGBM          : 0.000075 (t=0.047)
LOGME          : -0.002388 (t=-2.193)**
mom_minus1_0   : 0.000391 (t=0.291)
mom_minus12_minus1: 0.002900 (t=1.260)
mom_minus36_minus12: -0.001459 (t=-1.602)
mom_minus11_minus2: -0.000806 (t=-0.411)
turnover       : -0.004736 (t=-4.064)***
proxy          : -0.001105 (t=-1.564)
proxy*cgo      : -0.001059 (t=-1.369)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [138]:
i = 2
print(proxy_cols[i])
print(result_list_3_1[i])

retvol
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.006910 (t=1.064)
cgo            : 0.000202 (t=0.205)
LOGBM          : 0.000347 (t=0.227)
LOGME          : -0.002626 (t=-2.475)**
mom_minus1_0   : 0.000180 (t=0.128)
mom_minus12_minus1: 0.002417 (t=1.097)
mom_minus36_minus12: -0.001158 (t=-1.363)
mom_minus11_minus2: -0.000311 (t=-0.166)
turnover       : -0.004578 (t=-4.011)***
proxy          : -0.000766 (t=-0.930)
proxy*cgo      : -0.000802 (t=-1.160)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [139]:
i = 3
print(proxy_cols[i])
print(result_list_3_1[i])

age
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.006916 (t=1.069)
cgo            : 0.000520 (t=0.502)
LOGBM          : 0.000590 (t=0.382)
LOGME          : -0.002681 (t=-2.558)**
mom_minus1_0   : 0.000106 (t=0.081)
mom_minus12_minus1: 0.002307 (t=1.109)
mom_minus36_minus12: -0.001438 (t=-1.643)
mom_minus11_minus2: -0.000376 (t=-0.203)
turnover       : -0.004798 (t=-4.272)***
proxy          : 0.000133 (t=0.206)
proxy*cgo      : -0.000169 (t=-0.299)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [140]:
i = 4
print(proxy_cols[i])
print(result_list_3_1[i])

disp
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.006819 (t=1.067)
cgo            : 0.000207 (t=0.201)
LOGBM          : 0.000586 (t=0.371)
LOGME          : -0.002514 (t=-2.394)**
mom_minus1_0   : 0.000231 (t=0.173)
mom_minus12_minus1: 0.001894 (t=0.873)
mom_minus36_minus12: -0.001354 (t=-1.533)
mom_minus11_minus2: -0.000004 (t=-0.002)
turnover       : -0.004377 (t=-3.908)***
proxy          : 0.001227 (t=1.995)**
proxy*cgo      : 0.000709 (t=0.922)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [141]:
i = 5
print(proxy_cols[i])
print(result_list_3_1[i])

CFVOL
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.006903 (t=1.063)
cgo            : 0.000349 (t=0.335)
LOGBM          : 0.000670 (t=0.426)
LOGME          : -0.002683 (t=-2.503)**
mom_minus1_0   : 0.000268 (t=0.204)
mom_minus12_minus1: 0.002472 (t=1.192)
mom_minus36_minus12: -0.001370 (t=-1.646)
mom_minus11_minus2: -0.000508 (t=-0.268)
turnover       : -0.004645 (t=-4.017)***
proxy          : 0.000003 (t=0.007)
proxy*cgo      : -0.000359 (t=-0.715)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



# 3.2 regression
R = α+ β1 ×CGO + β2 ×P ROX Y + β3 ×P ROX Y × CGO + β4 ×SCORE + β5 ×P ROX Y ×SCORE + β6 ×LOGBM + β7 × LOGME + β8 ×M OM (−1 , 0) + β9 ×M OM (−12 , −1) + β10 ×M OM (−36 , −12) + β11 ×T URNOV ER + $\epsilon$ 

In [142]:
result_list = []

df = data2.copy()

# calc results
for proxy in proxy_cols:
    data3 = df[["date", "asset", "future_return"] + other_factor_cols].copy()
    data3["proxy"] = df[proxy]
    data3["proxy*cgo"] = data3["proxy"] * data3["cgo"]
    data3["mom_12_1*proxy"] = data3["mom_minus12_minus1"] * data3["proxy"]
    # data3[""]
    var_cols = data3.drop(columns=["date", "asset", "future_return"]).columns.to_list()
    # print(var_cols)
    
    result_list.append(famaMacBeth(data3, var_cols))
    # break

# then print results

In [143]:
i = 0
print(proxy_cols[i])
print(result_list[i])

beta_MKT
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo, mom_12_1*proxy

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.007018 (t=1.074)
cgo            : 0.000517 (t=0.517)
LOGBM          : 0.000640 (t=0.416)
LOGME          : -0.002730 (t=-2.624)***
mom_minus1_0   : 0.000172 (t=0.129)
mom_minus12_minus1: 0.002253 (t=1.023)
mom_minus36_minus12: -0.001337 (t=-1.538)
mom_minus11_minus2: -0.000102 (t=-0.053)
turnover       : -0.004697 (t=-4.042)***
proxy          : 0.000114 (t=0.138)
proxy*cgo      : -0.000103 (t=-0.186)
mom_12_1*proxy : -0.000003 (t=-0.006)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [144]:
i = 1
print(proxy_cols[i])
print(result_list[i])

ivol
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo, mom_12_1*proxy

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.007134 (t=1.087)
cgo            : 0.000019 (t=0.017)
LOGBM          : 0.000033 (t=0.021)
LOGME          : -0.002321 (t=-2.165)**
mom_minus1_0   : 0.000288 (t=0.209)
mom_minus12_minus1: 0.002836 (t=1.156)
mom_minus36_minus12: -0.001478 (t=-1.595)
mom_minus11_minus2: -0.000733 (t=-0.376)
turnover       : -0.004746 (t=-4.054)***
proxy          : -0.001024 (t=-1.428)
proxy*cgo      : -0.000996 (t=-1.275)
mom_12_1*proxy : 0.000136 (t=0.211)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [145]:
i = 2
print(proxy_cols[i])
print(result_list[i])

retvol
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo, mom_12_1*proxy

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.007094 (t=1.080)
cgo            : 0.000174 (t=0.169)
LOGBM          : 0.000318 (t=0.207)
LOGME          : -0.002606 (t=-2.479)**
mom_minus1_0   : 0.000137 (t=0.096)
mom_minus12_minus1: 0.002640 (t=1.065)
mom_minus36_minus12: -0.001107 (t=-1.274)
mom_minus11_minus2: -0.000448 (t=-0.232)
turnover       : -0.004658 (t=-3.885)***
proxy          : -0.000762 (t=-0.954)
proxy*cgo      : -0.000862 (t=-1.248)
mom_12_1*proxy : 0.000389 (t=0.580)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [146]:
i = 3
print(proxy_cols[i])
print(result_list[i])

age
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo, mom_12_1*proxy

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.006819 (t=1.054)
cgo            : 0.000630 (t=0.606)
LOGBM          : 0.000601 (t=0.388)
LOGME          : -0.002612 (t=-2.519)**
mom_minus1_0   : 0.000080 (t=0.060)
mom_minus12_minus1: 0.002004 (t=0.949)
mom_minus36_minus12: -0.001502 (t=-1.722)*
mom_minus11_minus2: -0.000374 (t=-0.199)
turnover       : -0.004740 (t=-4.165)***
proxy          : 0.000196 (t=0.307)
proxy*cgo      : 0.000229 (t=0.331)
mom_12_1*proxy : -0.000965 (t=-1.246)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [147]:
i = 4
print(proxy_cols[i])
print(result_list[i])

disp
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo, mom_12_1*proxy

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.006803 (t=1.064)
cgo            : 0.000180 (t=0.174)
LOGBM          : 0.000612 (t=0.386)
LOGME          : -0.002503 (t=-2.365)**
mom_minus1_0   : 0.000302 (t=0.229)
mom_minus12_minus1: 0.002080 (t=0.948)
mom_minus36_minus12: -0.001376 (t=-1.556)
mom_minus11_minus2: -0.000250 (t=-0.134)
turnover       : -0.004467 (t=-3.974)***
proxy          : 0.001549 (t=2.010)**
proxy*cgo      : 0.000180 (t=0.214)
mom_12_1*proxy : 0.001295 (t=1.302)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



In [148]:
i = 5
print(proxy_cols[i])
print(result_list[i])

CFVOL
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover, proxy, proxy*cgo, mom_12_1*proxy

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.006884 (t=1.061)
cgo            : 0.000353 (t=0.342)
LOGBM          : 0.000588 (t=0.370)
LOGME          : -0.002714 (t=-2.501)**
mom_minus1_0   : 0.000254 (t=0.193)
mom_minus12_minus1: 0.002584 (t=1.239)
mom_minus36_minus12: -0.001358 (t=-1.605)
mom_minus11_minus2: -0.000605 (t=-0.317)
turnover       : -0.004644 (t=-4.001)***
proxy          : -0.000093 (t=-0.171)
proxy*cgo      : -0.000235 (t=-0.409)
mom_12_1*proxy : -0.000651 (t=-1.040)

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01



# 3.3 regression

R = α+ β1 ×CGO + β2 ×P ROX Y + β3 ×P ROX Y × CGO + β4 ×SCORE + β5 ×P ROX Y ×SCORE + β6 ×LOGBM + β7 × LOGME + β8 ×M OM (−1 , 0) + β9 ×M OM (−12 , −1) + β10 ×M OM (−36 , −12) + β11 ×T URNOV ER + $\epsilon$ 

In [149]:
result_list_3_3 = []
for proxy in proxy_cols:
    var_cols_2 = ["cgo", proxy, "proxy*cgo", "score", "proxy*score", 'LOGBM',
        'LOGME', 'mom_minus1_0', 'mom_minus12_minus1', 'mom_minus36_minus12',
        'mom_minus11_minus2', "turnover"]

    data4 = data.copy()
    data4["proxy*cgo"] = data4[proxy] * data4["cgo"]
    data4["proxy*score"] = data4[proxy] * data4["score"]

    result_list_3_3.append(famaMacBeth(data4, var_cols_2))
    



In [150]:
# data4.info()

In [151]:
for p, r in zip(proxy_cols, result_list_3_3):
    print(p)
    print(r)

beta_MKT
Fama-MacBet reg result
# sample period: 120
dep var: future_return
ind var: cgo, beta_MKT, proxy*cgo, score, proxy*score, LOGBM, LOGME, mom_minus1_0, mom_minus12_minus1, mom_minus36_minus12, mom_minus11_minus2, turnover

factor premium est.:
--------------------------------------------------------------------------------
intercept      : 0.006753 (t=1.038)
cgo            : 0.000013 (t=0.012)
beta_MKT       : -0.000040 (t=-0.046)
proxy*cgo      : 0.000002 (t=0.003)
score          : -0.003150 (t=-2.036)**
proxy*score    : 0.000451 (t=1.072)
LOGBM          : -0.002216 (t=-1.187)
LOGME          : -0.002814 (t=-2.679)***
mom_minus1_0   : 0.000395 (t=0.294)
mom_minus12_minus1: 0.002266 (t=1.020)
mom_minus36_minus12: -0.001363 (t=-1.476)
mom_minus11_minus2: -0.002274 (t=-1.102)
turnover       : -0.004699 (t=-4.087)***

--------------------------------------------------------------------------------
Significance: * p<0.1, ** p<0.05, *** p<0.01

ivol
Fama-MacBet reg result
# sample per